# Notes on Topics 29 - 32

## Topic 29: Decision Trees

### Intro to Decision Trees

### Entropy and Information Gain

### ID3 Classification Trees: Perfect Split with Information Gain Lab

### Building Trees using scikit-learn + Lab

### Hyperparameter Tuning and Pruning in Decision Trees + Lab

### Regression with CART Trees + Lab

### Regression Trees and Model Optimization Lab

## Topic 30: Ensemble Methods

### Random Forests

### Tree Ensembles and Random Forests Lab

### GridSearchCV + Lab

### Gradient Boosting and Weak Learners + Lab

### XGBoost

## Topic 31: Support Vector Machines

### Building an SVM using scikit-learn Lab

### The Kernel Trick + Lab

## Topic 32: ML Pipelines
- Pipelines create an efficient workflow to combine data manipulations, preprocessing, and modeling

[Pipelines](https://www.kdnuggets.com/2017/12/managing-machine-learning-workflows-scikit-learn-pipelines-part-1.html)

[Integrating Grid Search](https://www.kdnuggets.com/2018/01/managing-machine-learning-workflows-scikit-learn-pipelines-part-2.html)

### Pipelines in sklearn Lab

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# defining the sequence of actions to perform
pipe = Pipeline([('mms', MinMaxScaler()),
                 ('tree', DecisionTreeClassifier(random_state=123))])
# fit the model,
pipe.fit(X_train, y_train)
# score the model on test data
pipe.score(X_test, y_test)

# or implement GridSearch
grid = [{'tree__max_depth': [None, 2, 6, 10], 
         'tree__min_samples_split': [5, 10]}]
gridsearch = GridSearchCV(estimator=pipe, 
                          param_grid=grid, 
                          scoring='accuracy', 
                          cv=5)
gridsearch.fit(X_train, y_train)
gridsearch.score(X_test, y_test)

## Extra Notes
[scoring parameters](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)